## Лекция 2  BM5    

### Imports

In [1]:
import numpy as np
import pandas as pd
import re
from math import log
from sklearn.feature_extraction.text import CountVectorizer
from time import time

### Constants

In [2]:
k = 2.0
b = 0.75
trained_size = 1000
N = trained_size*2

In [3]:
def enum_sort(arr): # takes a list and returns a list of ids in the decreasing order of the values from the input
    return [x[0] for x in sorted(enumerate(arr), key=lambda x:x[1], reverse=True)]

### Pre-computed dataset-dependednt constants

In [4]:
questions = pd.read_csv('quora_question_pairs_rus.csv', index_col=0)

In [5]:
questions.head()

,question1,question2,is_duplicate
0,Какова история кохинор кох-и-ноор-бриллиант,"что произойдет, если правительство Индии украд...",0
1,как я могу увеличить скорость моего интернет-с...,как повысить скорость интернета путем взлома ч...,0
2,"почему я мысленно очень одинок, как я могу это...","найти остаток, когда математика 23 ^ 24 матема...",0
3,которые растворяют в воде быстро сахарную соль...,какая рыба выживет в соленой воде,0
4,астрология: я - луна-колпачок из козерога и кр...,Я тройная луна-козерог и восхождение в козерог...,1


__only some texts will be used, a part defined by trained_size constant above__

In [6]:
train_texts = questions[:trained_size]['question1'].tolist() + questions[:trained_size]['question2'].tolist()
train_texts

['Какова история кохинор кох-и-ноор-бриллиант',
 'как я могу увеличить скорость моего интернет-соединения, используя vpn',
 'почему я мысленно очень одинок, как я могу это решить',
 'которые растворяют в воде быстро сахарную соль метан и углеродный диоксид',
 'астрология: я - луна-колпачок из козерога и крышка, поднимающая то, что это говорит обо мне',
 'я должен купить tiago',
 'как я могу быть хорошим геологом?',
 'когда вы используете вместо',
 'компания motorola: могу ли я взломать мой чартер motorolla dcx3400',
 'метод определения разделения прорезей с использованием бипризма френеля',
 'как мне читать и находить комментарии к YouTube',
 'что может сделать физику легкой для изучения',
 'какой был ваш первый сексуальный опыт, как',
 'каковы законы об изменении вашего статуса от студенческой визы до зеленой карты в нас, как они сравниваются с иммиграционным законодательством в Канаде',
 'что означало бы президентство козыря для нынешних иностранных студентов-магистров по визе f1',
 

__define mean text length__

In [7]:
lens = [len(text.split()) for text in train_texts]
avgdl = sum(lens)/N
avgdl

9.201

__precompute a count matrix__
<br> rows - documents
<br> columns - words

In [8]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(train_texts)
count_matrix = X.toarray()
count_matrix.shape

(2000, 5905)

__precompute tfs__

In [9]:
tf_matrix = count_matrix / np.array(lens).reshape((-1, 1))
tf_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

get a vocabulary that has the same indexation as the rows of the count matrix

In [10]:
vocabulary = vectorizer.get_feature_names()
vocabulary[1000:1010]

['внедрять',
 'вносит',
 'вносить',
 'внутри',
 'во',
 'вода',
 'воде',
 'водой',
 'воды',
 'военные']

__get idfs__

a list of number of docs with a given word for each word

In [11]:
in_n_docs = np.count_nonzero(count_matrix, axis=0)
in_n_docs

array([ 1,  5, 11, ...,  1,  1,  1], dtype=int64)

In [12]:
def IDF_modified(word):
    word_id = vocabulary.index(word)
    n = in_n_docs[word_id]
    return log((N - n + 0.5) / (n + 0.5))

In [13]:
IDF_modified('воде')

6.6838614462772235

In [14]:
idfs = [IDF_modified(word) for word in vocabulary]
idfs[1000:1010]

[7.195187320178709,
 7.195187320178709,
 7.195187320178709,
 7.195187320178709,
 4.3904097651860585,
 6.6838614462772235,
 6.6838614462772235,
 7.195187320178709,
 6.6838614462772235,
 7.195187320178709]

## Функция ранжирования bm25

Для обратного индекса есть общепринятая формула для ранжирования *Okapi best match 25* ([Okapi BM25](https://ru.wikipedia.org/wiki/Okapi_BM25)).    
Пусть дан запрос $Q$, содержащий слова  $q_1, ... , q_n$, тогда функция BM25 даёт следующую оценку релевантности документа $D$ запросу $Q$:

$$ score(D, Q) = \sum_{i}^{n} \text{IDF}(q_i)*\frac{TF(q_i,D)*(k+1)}{TF(q_i,D)+k(1-b+b\frac{l(d)}{avgdl})} $$ 
где   
>$TF(q_i,D)$ - частота слова $q_i$ в документе $D$      
$l(d)$ - длина документа (количество слов в нём)   
*avgdl* — средняя длина документа в коллекции    
$k$ и $b$ — свободные коэффициенты, обычно их выбирают как $k$=2.0 и $b$=0.75   
$$$$
$\text{IDF}(q_i)$ - это модернизированная версия IDF: 
$$\text{IDF}(q_i) = \log\frac{N-n(q_i)+0.5}{n(q_i)+0.5},$$
>> где $N$ - общее количество документов в коллекции   
$n(q_i)$ — количество документов, содержащих $q_i$

### implement tf part of the formula

In [15]:
def modify_tf(tf_value, doc_index):
    l = lens[doc_index]
    return (tf_value * (k + 1.0))/(tf_value + k * (1.0 - b + b * (l/avgdl)))

def modify_tf_matrix(tf_matrix): 
    enumed =  np.ndenumerate(tf_matrix)
    for i, tf_value in enumed:
        doc_index = i[0]
        tf_matrix[i] = modify_tf(tf_value, doc_index)
    return tf_matrix

In [16]:
modified_tf_matrix = modify_tf_matrix(tf_matrix)
modified_tf_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

__clean the query from punctuation, 
<br>get to lower, 
<br>remove out of vocabulary words__

In [17]:
def preprocess_query(query_text):
    low = query_text .lower()
    stripped = re.sub('\.|,|#|$|%|\\|\'|\(|\)|-|\+|\*|/|\:|;|<|>|=|\?|\[|\]|@|^|_|`|{|}|~|!', '', low)
    words = stripped.split()
    query_modified = list(set(words).intersection(set(vocabulary)))  
    return query_modified

In [18]:
q = 'БЛЯТЬ!111 Я ЗАЕБАЛАСЬ ВОороны!22 ебутся в воде !11'
preprocess_query(q)

['11', 'воде']

### __Задача 1__:    
Напишите два поисковика на *BM25*. Один через подсчет метрики по формуле для каждой пары слово-документ, второй через умножение матрицы на вектор. 

Сравните время работы поиска на 100к запросах. В качестве корпуса возьмем 
[Quora question pairs](https://www.kaggle.com/loopdigga/quora-question-pairs-russian).

### define two bm25 implementations

In [19]:
### реализуйте эту функцию ранжирования векторно
def bm25_vector(query):
    vector = np.array(vectorizer.transform([query]).todense())[0]
    binary_vector = np.vectorize(lambda x: 1.0 if x != 0.0 else 0.0)(vector) ## neutralizes duplictes in the query (non-lineraity)
    idfs_from_query = np.array(idfs)*np.array(binary_vector)
    return modified_tf_matrix.dot(idfs_from_query) ## bm25 близость для каждого документа

In [20]:
### реализуйте эту функцию ранжирования итеративно
def bm25_iter(query):
    query_words = preprocess_query(query)
    relevance = []
    for i, doc in np.ndenumerate(tf_matrix):
        doc_index = i[0]
        doc_bm25 = 0.0
        for word in set(query_words): ## set neutralizes duplictes in the query
            word_index = vocabulary.index(word)
            tf_value = tf_matrix[(doc_index, word_index)]
            doc_bm25 += idfs[word_index] * modify_tf(tf_value, doc_index)
        relevance.append(doc_bm25)
    return relevance

__Compare performance__

In [24]:
start = time()
print(bm25_vector('если честно, мне кажется, что мой итератиынй алгоритм работает очень плохо 11 !!'))
print('TIME: ' + str(time() - start))

[0.         0.         0.79281002 ... 0.28146463 0.         0.        ]
TIME: 0.02001476287841797


In [ ]:
start = time()
print(bm25_iter('если честно, мне кажется, что мой итератиынй алгоритм работает очень плохо 11 !!'))
print('TIME: ' + str(time() - start))

### __Задача 2__:    



Выведите 10 первых результатов и их близость по метрике BM25 по запросу **рождественские каникулы** на нашем корпусе  Quora question pairs. 

### __Задача 3__:    

Посчитайте точность поиска при 
1. BM25, b=0.75 
2. BM15, b=0 
3. BM11, b=1